In [2]:
import numpy as np
import pandas as pd
import re,math
import concurrent.futures
import multiprocessing
import logging

# 设置日志级别
logging.basicConfig(level=logging.DEBUG)

# 设置函数运行计时用的修饰器
def timeit(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} took {end - start:.6f} seconds')
        return result
    return wrapper


data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'583drugs39cell Zscore行标准化 synergy.xlsx')
drug_names = list(data1.index)

# 合并drugA和drugB为一列作为索引
# 顺序
temp1 = [] # 分子指纹合并值 【【】，【】，【】，】
index_1 = [] # 对应的名字列表 【【】，【】，【】，】
# 逆序
temp2 = []
index_2 = []
for index,row in data2.iterrows():
    ct = list((np.array(data1.loc[row["DrugA"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugB"],:]))))
    temp1.append(ct)
    index_1.append((row["DrugA"],row["DrugB"]))
    ct = list((np.array(data1.loc[row["DrugB"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugA"],:]))))
    temp2.append(ct)
    index_2.append((row["DrugB"],row["DrugA"]))

/root/miniconda3/envs/jupyter-env/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:

# # 两组数据的tanimoto计算
# def tanimoto(p,q):
#     tep1=0
#     tep2=0
#     lenthp=int(len(p))
#     for i in range(lenthp):
#         a=p[i]
#         b=q[i]
#         if (a==1)|(b==1): # 并
#             tep1=tep1+1
#         if (a==1)&(b==1): # 交
#             tep2=tep2+1
#     c=round((tep2 / tep1),4)      #取值4位数
#     return c

# # 相互之间做一次tanimoto
# temp=[]
# Tanimoto_index_dist = {} # {顺序药物对：(实际药物名字1，实际药物名字2，值)}
# for i,j,i_n,j_n in zip(temp1,temp2,index_1,index_2):
#     tmp=[]
#     a = np.array(i) #取出一分子指纹 转换列表为向量
#     b = np.array(j)
#     for k,m,k_n,m_n in zip(temp1,temp2,index_1,index_2): # 遍历每一个药物对的分子指纹
#         c = np.array(k)
#         d = np.array(m)
#         tp1 = tanimoto(a,c)
#         tp2 = tanimoto(a,d)
#         tp3 = tanimoto(b,c)
#         tp4 = tanimoto(b,d)
#         tps = [tp1, tp2, tp3, tp4]
#         index_list = [(i_n,k_n,tp1),(i_n,m_n,tp2),(j_n,k_n,tp3),(j_n,m_n,tp4)]
#         index_location = tps.index(max(tps))
#         tmp.append(max(tps))
#         Tanimoto_index_dist[f"{i_n},{k_n}"] = index_list[index_location]
#     temp.append(tmp)
# Tanimoto=pd.DataFrame(temp,columns=index_1,index=index_1) # 数据框 dataframe

In [4]:
# 两组数据的tanimoto计算
def tanimoto(p,q):
    tep1=0
    tep2=0
    lenthp=int(len(p))
    for i in range(lenthp):
        a=p[i]
        b=q[i]
        if (a==1)|(b==1): # 并
            tep1=tep1+1
        if (a==1)&(b==1): # 交
            tep2=tep2+1
    c=round((tep2 / tep1),4)      #取值4位数
    return c

def compute_tanimoto(i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx):
    tmp=[]
    a = np.array(i) #取出一分子指纹 转换列表为向量
    b = np.array(j)
    for k,m,k_n,m_n in zip(temp1,temp2,index_1,index_2): # 遍历每一个药物对的分子指纹
        c = np.array(k)
        d = np.array(m)
        tp1 = tanimoto(a,c)
        tp2 = tanimoto(a,d)
        tp3 = tanimoto(b,c)
        tp4 = tanimoto(b,d)
        tps = [tp1, tp2, tp3, tp4]
        index_list = [(i_n,k_n,tp1),(i_n,m_n,tp2),(j_n,k_n,tp3),(j_n,m_n,tp4)]
        index_location = tps.index(max(tps))
        tmp.append(max(tps))
        Tanimoto_index_dist[f"{i_n},{k_n}"] = index_list[index_location]
    return idx, tmp

temp = [None] * len(temp1)
Tanimoto_index_dist = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 提交任务时使用 enumerate ，并将索引值作为参数传递给任务函数
    futures = [executor.submit(compute_tanimoto, i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx)
               for idx, (i,j,i_n,j_n) in enumerate(zip(temp1,temp2,index_1,index_2))]
    for future in concurrent.futures.as_completed(futures):
        idx, result = future.result()
        temp[idx] = result
Tanimoto = pd.DataFrame(temp, columns=index_1, index=index_1)

In [5]:
# Tanimoto

In [6]:
# 获取columns数组 - 处理后的
ttpp = []
for i in list(Tanimoto.columns):
    ttpp.append(str(i)[1:-1]) # 去除首尾括号的药物对名字
data2 = pd.DataFrame(data2.iloc[:,2:].T.values, columns=list(ttpp)) # 药物对协同得分
columns = data2.columns # 修改后的药物对名字

In [7]:
# 删去对应行后的字典
dict_lines = {} # {键1：值1} 谷本系数字典
count = 0
ct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,ct]) # 每一列谷本系数 [:,1] 第一列的每一行 实际就是第一列
    y.pop(count)
    count += 1
    dict_lines[columns[ct]] = y # 谷本系数药物对的名字做键1，y是值1
    ct += 1

In [8]:
# # 高相似多少
# coum=200

# # 分离x y数据构造字典
# dict_columns = {}  # {a1:data,a2:data} 药物对名字-对应数据
# for i in columns:

#     # b 要预测协同得分那一列
#     y = data2.loc[:, i]

#     # A 权重
#     x = data2.drop(columns=i)  # 其他所有的列

#     # 高相似的    取高相似的药物对名字max1和对应的值max5
#     tp = []
#     for j in columns:
#         if i != j:
#             tp.append(j)
#     df = pd.DataFrame(dict_lines[i], index = tp)
#     df.sort_values(by=0, inplace=True, ascending=True)
#     max_l = df.index[-coum:]
#     max_5x = pd.DataFrame([data2.loc[:, j] for j in max_l]).T

#     # 存储 对应数据
#     data = {
#         "x": x,  # 其他所有的列 数据表，含药物对名字
#         "y": y,  # 要预测协同得分的那一列 含药物对名字
#         # 下面两个是对应的
#         "max_relevance": max_l,  # 高相似名字
#         "max_5x": max_5x  # 前多少高相似邻居对应的列
#     }
#     dict_columns[i] = data

# dict_columns_split = {} # 高相似字典
# for i in dict_columns: # 第几列 i 是药物对名字
#     count = 0 # 第几行 int
#     for j in list(dict_columns[i]['y'].index): # 35个索引 0~34 数字（字符串-str） 【0，1，2，3.。。。】
#         dict_columns_split[f'{i},{j}']={} # 空字典 键-空字典
#         dict_columns_split[f'{i},{j}']['y_line']=dict_columns[i]["y"][count]
#         dict_columns_split[f'{i},{j}']['max_x_line']=dict_columns[i]["max_5x"].iloc[count,:].values
#         count += 1

In [9]:
# 高相似多少
coum=5
# 分离x y数据构造字典
dict_columns_split = {} # 高相似字典
# 计算x数据
x = data2.drop(columns=columns)  # 其他所有的列
for i in columns:
    # b 要预测协同得分那一列
    y = data2.loc[:, i]
    # A 权重
    x_i = x.copy()
    x_i['y'] = y

    # 高相似的    取高相似的药物对名字max1和对应的值max5
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_lines[i], index = tp)
    df.sort_values(by=0, inplace=True, ascending=True)
    max_l = df.index[-coum:]
    max_5x = pd.DataFrame([data2.loc[:, j] for j in max_l]).T

    # 存储 对应数据
    count = 0 # 第几行 int
    for j in list(y.index): # 35个索引 0~34 数字（字符串-str） 【0，1，2，3.。。。】
        dict_columns_split[f'{i},{j}']={} # 空字典 键-空字典
        dict_columns_split[f'{i},{j}']['y_line']=y[count]
        dict_columns_split[f'{i},{j}']['max_x_line']=max_5x.iloc[count,:].values
        count += 1

In [10]:
dict_columns_split["'SUNITINIB', 'MK-4827',32"]['max_x_line']

array([-0.14583069,  0.17624582,  0.40102622, -0.4231079 ,  0.06873453])

In [1063]:
# # 高相似谷本系数字典 (删去对应行后的字典)
# dict_high_lines = {} 
# count = 0
# cct = 0
# for i in Tanimoto.index: # 循环遍历谷本系数名字
#     y = list(Tanimoto.iloc[:,cct])
#     y.pop(count) # 去除相同药物对的谷本系数
#     count += 1
#     dict_high_lines[str(i)[1:-1]] = y 
#     tp = []
#     for j in columns: 
#         if str(i)[1:-1] != j:
#             tp.append(j)
#     df = pd.DataFrame(dict_high_lines[str(i)[1:-1]], index=tp) 
#     df.sort_values(by=0 , inplace=True, ascending=True) 

#     max_l = df.index[-coum:]  # 
#     tp = [] 
#     for j in max_l:
#         tp.append(Tanimoto.iloc[list(columns).index(j), :])
#     max_5y = list(pd.DataFrame(tp).T.iloc[[list(columns).index(str(i)[1:-1])]].values)[0]

#     data = {
#         "y": y,
#         "max_relevance": max_l,
#         "high_5": max_5y
#     }
#     dict_high_lines[str(i)[1:-1]] = data
#     cct += 1
# dict_high_lines_split = {}
# for i in dict_high_lines: 
#     count = 0
#     for j in dict_high_lines[i]['max_relevance']: 
#         dict_high_lines_split[f'{i},{j}']=dict_high_lines[i]['high_5'][count]
#         count += 1
# TTT = [] # 值列表
# for i in dict_high_lines_split:
#     TTT.append(dict_high_lines_split[i])

In [1064]:
TTT = [] # 值列表
cct = 0
for i in Tanimoto.index: 
    y = list(Tanimoto.iloc[:,cct])
    y.pop(cct) 
    tp_columns = list(columns.copy())
    tp_columns.pop(cct)
    df = pd.DataFrame(y, index=tp_columns)
    df.sort_values(by=0 , inplace=True, ascending=True) 
    max_l = df.index[-coum:]  
    # 计算max_5y
    max_5y = []
    for j in max_l:
        max_5y.append(Tanimoto.iloc[list(columns).index(j), cct])
    # 将max_5y加入TTT
    TTT += max_5y
    # 修改cct变量的值
    cct += 1

In [1065]:
# TTT[-210]

In [1066]:
# def alpha_pcc(t_coum, a):
#     #求PCC
#     alpha = []
#     result_pcc = []
#     lin = np.linspace(1,3,401)
#     pcc_split = []
#     for alp in lin:
#         pre = []
#         for elem in range(39):
#             pp = []
#             for j in TTT[(coum * a - t_coum):(coum * a)]: 
#                 pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))
#             pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
#         pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
#     result_pcc.append(np.max(pcc_split))
#     alpha.append(lin[pcc_split.index(np.max(pcc_split))])
#     pcc=result_pcc[0]
#     return alpha, pcc

In [1067]:
# 单进程
def alpha_pcc(t_coum, a, lin, linename):
    alpha = []
    result_pcc = []
    pcc_split = []
    for alp in lin:
        pre = []
        for elem in range(39):
            pp = [] 
            if TTT[(coum * a - t_coum):(coum * a)] == []:
                print(1111111111111)
            for j in TTT[(coum * a - t_coum):(coum * a)]: 
                pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))
            pp = np.array(pp)  # 将 pp 转换为 numpy 数组
            pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
        pre = np.array(pre)  # 将 pre 转换为 numpy 数组
        pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
    result_pcc.append(np.max(pcc_split))
    alpha.append(lin[pcc_split.index(np.max(pcc_split))])
    pcc=result_pcc[0]
    del result_pcc, pcc_split, pre, pp
    return alpha, pcc

In [1068]:
# def alp_mse_rmse(alpha, t_coum, a):
#     r1 = []
#     alp = alpha[0]
#     for elem in range(39):
#         pp = []
#         for j in TTT[(coum * a - t_coum):(coum * a)]: 
#             pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  
#         r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) - dict_columns_split[f"{linename},{elem}"]['y_line'])
#     mse=np.mean(np.square(r1))
#     rmse=math.sqrt(mse)
#     return alp, mse, rmse

In [1069]:
# 单进程
def alp_mse_rmse(alpha, t_coum, a, linename):
    r1 = []  # 初始化 r1 为空列表
    alp = alpha[0]
    for elem in range(39):
        pp = []
        for j in TTT[(coum * a - t_coum):(coum * a)]: 
            pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  
        r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) - dict_columns_split[f"{linename},{elem}"]['y_line'])
    r1 = np.array(r1)  # 将 r1 转换为 numpy 数组
    mse = np.mean(np.square(r1))
    rmse = math.sqrt(mse)
    del r1, pp,
    return alp, mse, rmse

In [1070]:
# # 单进程
# Name = []
# a = 1
# # 加载保存的数组
# lin = np.load("lin.npy")
# for linename in columns:
#     PCC = 0
#     for t_coum in range(2, coum+1, 1):   
#         #求PCC
#         alpha, pcc = alpha_pcc(t_coum, a, lin, linename)
#         #求MSE
#         alp, mse, rmse = alp_mse_rmse(alpha, t_coum, a, linename)
#         if PCC == 0:
#             Name_coum, Name_mse, Name_pcc, Name_alp, PCC, Name_rmse = t_coum, mse, pcc, alp, pcc, rmse
#         if pcc > PCC:
#             Name_coum, Name_mse, Name_pcc, Name_alp, PCC, Name_rmse = t_coum, mse, pcc, alp, pcc, rmse
#         print("name:{},coum:{:0d},alp={},mse={},pcc={},rmse={}".format(linename, t_coum, alp, mse, pcc,rmse))
#     a = a+1
#     Name.append([linename, Name_coum, Name_alp, Name_mse, Name_pcc, Name_rmse])
#     del linename, Name_coum, Name_alp, Name_mse, Name_pcc, Name_rmse, t_coum, mse, pcc, alp, rmse
# pd.DataFrame(Name).to_csv('pcc_cpu_test_one.csv', header=None, index=False)

In [1072]:
# 单进程 分函数
def calculate_parameters(linename, a):
    PCC = 0
    lin = np.linspace(1,3,401)
    for t_coum in range(2, coum+1, 1):   
        # 求PCC
        alpha, pcc = alpha_pcc(t_coum, a, lin, linename)
        # 求MSE
        alp, mse, rmse = alp_mse_rmse(alpha, t_coum, a, linename)
        if PCC == 0:
            Name_coum, Name_mse, Name_pcc, Name_alp, PCC, Name_rmse = t_coum, mse, pcc, alp, pcc, rmse
        if pcc > PCC:
            Name_coum, Name_mse, Name_pcc, Name_alp, PCC, Name_rmse = t_coum, mse, pcc, alp, pcc, rmse
        # print("name:{},coum:{:0d},alp={},mse={},pcc={},rmse={}".format(linename, t_coum, alp, mse, pcc,rmse))
    return [linename, Name_coum, Name_alp, Name_mse, Name_pcc, Name_rmse]

@timeit
def one_thread():
    a = 1
    results = []
    for linename in columns:
        result = calculate_parameters(linename, a)
        results.append(result)
        del result
        a += 1
    return results

# 单进程执行
results = one_thread()
# 将结果保存到 CSV 文件中
pd.DataFrame(results).to_csv('pcc_cpu_test_one.csv', header=None, index=False)

name:'MK-5108', 'SORAFENIB',coum:2,alp=3.0,mse=1.090498180475555,pcc=0.3620820743983432,rmse=1.0442692088132997
name:'MK-5108', 'SORAFENIB',coum:3,alp=3.0,mse=0.9272598178486342,pcc=0.4231142457102548,rmse=0.9629433097792591
name:'MK-5108', 'SORAFENIB',coum:4,alp=3.0,mse=0.8384259376227242,pcc=0.4529898145021846,rmse=0.9156560149000956
name:'MK-5108', 'SORAFENIB',coum:5,alp=3.0,mse=0.8273553604482723,pcc=0.45589410641038647,rmse=0.9095907653710389
name:'VINORELBINE', 'SUNITINIB',coum:2,alp=1.0,mse=0.9343433897871151,pcc=0.38951726431418193,rmse=0.9666143956030838
name:'VINORELBINE', 'SUNITINIB',coum:3,alp=1.0,mse=0.8481594441297248,pcc=0.4311388090670471,rmse=0.9209557232189421
name:'VINORELBINE', 'SUNITINIB',coum:4,alp=1.0,mse=0.8449990904643294,pcc=0.41401600709782493,rmse=0.9192383208201937
name:'VINORELBINE', 'SUNITINIB',coum:5,alp=1.0,mse=0.8871201069766909,pcc=0.35774197819351256,rmse=0.941870536207971
name:'SUNITINIB', 'MK-8776',coum:2,alp=3.0,mse=1.0989472371968743,pcc=0.308375

In [942]:
# Name = []
# a = 1
# for linename in columns:
#     PCC = 0
#     MSE = 1000
#     result = []
#     for coum in range(2, 201, 1):   
#         #求PCC
#         alpha = []
#         result_pcc = []
#         lin = np.linspace(1,3,401)
#         pcc_split = []
#         for alp in lin:
#             pre = []
#             for elem in range(39):
#                 pp = []
#                 for j in TTT[(200 * a - coum):(200 * a)]: 
#                     pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))
#                 pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-coum:]))
#             pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
#         result_pcc.append(np.max(pcc_split))
#         alpha.append(lin[pcc_split.index(np.max(pcc_split))])
#         pcc=result_pcc[0]
#         #求MSE
#         r1 = []
#         alp = alpha[0]
#         for elem in range(39):
#             pp = []
#             for j in TTT[(200 * a - coum):(200 * a)]: 
#                 pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  
#             r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-coum:]) - dict_columns_split[f"{linename},{elem}"]['y_line'])
#         mse=np.mean(np.square(r1))
#         rmse=math.sqrt(mse)
#         if pcc > PCC:
#             Name_coum, Name_mse, Name_pcc, Name_alp, PCC, Name_rmse = coum, mse, pcc, alp, pcc, rmse
#         elif PCC == 0:
#             Name_coum, Name_mse, Name_pcc, Name_alp, PCC, Name_rmse = t_coum, mse, pcc, alp, pcc, rmse
#         print("name:{},coum:{:0d},alp={},mse={},pcc={},rmse={}".format(linename, coum, alp, mse, pcc,rmse))
#     a = a+1
#     Name.append([linename, Name_coum, Name_alp, Name_mse, Name_pcc, Name_rmse])
# pd.DataFrame(Name).to_csv('Tanimotomax synergy anlieduqu zscore result all.csv', header=None, index=False)